# Chance-constrained programming: a portfolio optimization example

In [1]:
using LinearAlgebra
using JuMP
using HiGHS, Gurobi
using Distributions
using Random

*Adapted from Stephen Boyd and Lieven Vandenberghe, "Convex Optimization", Cambridge University Press, 2004, Section 4.7.6, p. 187.*

We consider an investiment portfolio with $n$ assets and random returns, except the last one. We assume that the $n-1$ first returns follow a multivariate distribution with known mean and covariance matrix. The last asset is a risk-free investment product, with a fixed return.

The assets characteristics are detailed below.

In [2]:
n = 4   # number of assets

# The last asset is a risk-free asset, with a null variance.
# We ignore it in the covariance matrix in order to keep it positive definite.
μ = [.12 ; .10 ; .07 ; .03]
Σ = [ 4e-2  6e-3 -4e-3  ;
      6e-3  1e-2  0.0  ;
      -4e-3  0.0 2.5e-3 ]

# We create une multivariate normal of mean μ and covariance matrix Σ
d = MvNormal(μ[1:n-1], Σ)

FullNormal(
dim: 3
μ: [0.12, 0.1, 0.07]
Σ: [0.04 0.006 -0.004; 0.006 0.01 0.0; -0.004 0.0 0.0025]
)


The following function estimates by simulation the probability to have a negative return and the resulting expected shortfall.

In [3]:
function expectedshortfall(p:: Vector, d:: Distribution, M:: Int = 1000000)
    
    loss = 0
    vloss = 0
    for i = 1:M
        ξ = [rand(d); μ[n]]
        ret = dot(p, ξ)
        if ret < 0
            loss += 1
            vloss += ret
        end
    end

    return loss/M, vloss/loss
    
end

expectedshortfall (generic function with 2 methods)

## Portfolio with uniform repartition

We first consider the naive strategy where the same amount is invested in each asset.

In [4]:
# Expected return with a uniform repartition.
p = ones(n)./n
er = sum(p[i]*μ[i] for i = 1:n)

0.08000000000000002

We now compute the loss probability and the average loss when a loss occurs.

In [5]:
ploss, eshortfall = expectedshortfall(p, d)
println("Loss probability: ", ploss)
println("Expected shortfall: ", eshortfall)

Loss probability: 0.089107
Expected shortfall: -0.027641618679356984


## Optimal decision without loss constraint

We now aim to maximize the expected return, without any consideration for the potential loss. We allow to sell some assets in favor of other ones.

In [6]:
m = Model(HiGHS.Optimizer)

@variable(m, p[1:n] >= -0.1)
@constraint(m, sum(p[i] for i = 1:n) <= 1)

@objective(m, Max, sum(p[i]*μ[i] for i = 1:n))

println(m)

Max 0.12 p[1] + 0.1 p[2] + 0.07 p[3] + 0.03 p[4]
Subject to
 p[1] + p[2] + p[3] + p[4] <= 1.0
 p[1] >= -0.1
 p[2] >= -0.1
 p[3] >= -0.1
 p[4] >= -0.1



In [7]:
optimize!(m)

Presolving model
1 rows, 1 cols, 1 nonzeros
0 rows, 0 cols, 0 nonzeros
Presolve : Reductions: rows 0(-1); columns 0(-4); elements 0(-4) - Reduced to empty
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Objective value     :  1.3600000000e-01
HiGHS run time      :          0.00


In [8]:
println("Loss probability: ", value.(p))

Loss probability: [1.3, -0.1, -0.1, -0.1]


Not surprisingly, we invest everything is the asset having the highest return rate, even if it means borrowing from the lower return rate asset.

In [9]:
println("Expected return: ", objective_value(m))

Expected return: 0.13599999999999998


The loss probability is however close to 30%, and the average lost amount is significantly more important if a loss occurs.

In [10]:
ploss, eshortfall = expectedshortfall(value.(p), d)
println("Loss probability: ", ploss)
println("Expected shortfall: ", eshortfall)

Loss probability: 0.29986
Expected shortfall: -0.1646197032693583


## Optimal decision with loss constraint

We add the constraint that we want to limit the risk by accepting a loss with a maximum probability of 0.05.

In [11]:
maxloss = 0 # maximum tolerated loss

α = 0.95
z = 1/quantile(Normal(0,1), α)

0.6079568319117693

We build the second-order cone constraint corresponding to the joint chance constraint.

Unfortunately, HiGHS does not support second-order cone constraint. We first switch to Gurobi and then add the constraint.

In [12]:
set_optimizer(m, Gurobi.Optimizer)

# || x || <= t, t >= 0
# https://jump.dev/JuMP.jl/stable/reference/constraints/#JuMP.SecondOrderCone
@constraint(m, [z*(-maxloss+sum(μ[i]*p[i] for i = 1:n)); (Σ^0.5)*p[1:n-1]] in SecondOrderCone())

println("Model:\n", m)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-09-27
Model:
Max 0.12 p[1] + 0.1 p[2] + 0.07 p[3] + 0.03 p[4]
Subject to
 p[1] + p[2] + p[3] + p[4] <= 1.0
 [0.0729548198294123 p[1] + 0.06079568319117693 p[2] + 0.04255697823382386 p[3] + 0.018238704957353077 p[4], 0.19827331129330036 p[1] + 0.02038945638178348 p[2] - 0.016491334004420634 p[3], 0.02038945638178348 p[1] + 0.09787182302877327 p[2] + 0.002318690466585327 p[3], -0.016491334004420634 p[1] + 0.002318690466585327 p[2] + 0.04714509070173485 p[3]] in MathOptInterface.SecondOrderCone(4)
 p[1] >= -0.1
 p[2] >= -0.1
 p[3] >= -0.1
 p[4] >= -0.1



We now optimize it.

In [13]:
optimize!(m)

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 5 rows, 8 columns and 21 nonzeros
Model fingerprint: 0x9ff28c3e
Model has 1 quadratic constraint
Coefficient statistics:
  Matrix range     [2e-03, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [3e-02, 1e-01]
  Bounds range     [1e-01, 1e-01]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.01s
Presolved: 5 rows, 8 columns, 21 nonzeros
Presolved model has 1 second-order cone constraint
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 1.000e+01
 Factor NZ  : 1.500e+01
 Factor Ops : 5.500e+01 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   1.11301574e-01  5.68875345e-02  3.97e-01 1.28e-01  3.28e-02     0s
   1   9.26826200e-02  1.04042953e-01  2.41e-02 1.64e-02  6.54e-03     0s
  

In [14]:
println("Optimal portfolio: ", value.(p))
println("Expected return: ", objective_value(m))

Optimal portfolio: [0.1814209102161574, 0.3410104123091543, 0.577568613079467, -0.09999996820959088]
Expected return: 0.0933013543261293


Without any surprise, the expected return is less than without the loss constraint, but is still higher than with the uniform repartition. We can also see that we use the risk-free asset as leverage to buy riskier assets. The risk to lose money is limited to 5%, as desired, and is less than any other strategy. Interestingly, the associated average loss is also the smallest one.

In [15]:
ploss, eshortfall = expectedshortfall(value.(p), d)
println("Loss probability: ", ploss)
println("Expected shortfall: ", eshortfall)

Loss probability: 0.049803
Expected shortfall: -0.02359441696806257
